In [1]:
import pandas as pd
from collections import namedtuple

In [2]:
from storage import session_scope, Movie, MovieAward, RTReview

In [3]:
CATEGORY = 'Best Motion Picture of the Year'

In [4]:
with session_scope() as session:
    movies_df = pd.read_sql(session.query(Movie).statement, session.bind)
    movie_awards_df = pd.read_sql(session.query(MovieAward).statement, session.bind)
    reviews_df = pd.read_sql(session.query(RTReview).statement, session.bind)

In [5]:
movies_df.head()

,imdb_id,title,release_year,directors,actors,runtime,genres,countries,imdb_rating,imdb_votes,box_office,rt_url,rt_tomato_score,rt_audience_score
0,tt0440963,The Bourne Ultimatum,2007,[Paul Greengrass],"[Matt Damon, Julia Stiles, David Strathairn, S...",115 min,"[Action, Mystery, Thriller]","[USA, Germany, France, Spain]",8.0,573745.0,"$227,400,000",http://www.rottentomatoes.com/m/bourne_ultimatum/,NaN,NaN
1,tt0978759,Frozen River,2008,[Courtney Hunt],"[Melissa Leo, Misty Upham, Charlie McDermott, ...",97 min,"[Crime, Drama]",[USA],7.2,23348.0,"$2,061,019",http://www.rottentomatoes.com/m/frozen_river/,NaN,NaN
2,tt2660888,Star Trek Beyond,2016,[Justin Lin],"[Chris Pine, Zachary Quinto, Karl Urban, Zoe S...",122 min,"[Action, Adventure, Sci-Fi, Thriller]","[USA, China, United Arab Emirates, Canada]",7.1,204882.0,"$158,804,470",http://www.rottentomatoes.com/m/star_trek_beyond/,NaN,NaN
3,tt0315733,21 Grams,2003,[Alejandro G. Iñárritu],"[Sean Penn, Naomi Watts, Danny Huston, Carly N...",124 min,"[Crime, Drama, Thriller]",[USA],7.7,209946.0,"$16,162,920",http://www.rottentomatoes.com/m/21_grams/,NaN,NaN
4,tt5599918,Pearl,2016,[Patrick Osborne],"[Nicki Bluhm, Kelley Stoltz, Emma Grace Eisenm...",6 min,"[Animation, Short, Music]",[USA],7.0,1593.0,N/A,N/A,NaN,NaN


In [6]:
reviews_df.head()

,type,movie_imdb_id,reviewer_url,reviewer_name,review_date,fresh,original_score,review_text
0,critic,tt0308644,/critic/david-ansen/,David Ansen Newsweek,2018-03-12,True,NaN,"Mingling reality and fantasy, Forster has give..."
1,critic,tt0308644,/critic/will-self/,Will Self London Evening Standard,2017-12-26,False,NaN,"Wooden, yes, and saccharine, too. Oh, and did ..."
2,critic,tt0308644,/critic/nell-minow/,Nell Minow Common Sense Media,2010-12-24,True,0.6,"Not about Peter Pan, and not a kids' movie."
3,critic,tt0308644,/critic/felix-vasquez-jr/,Felix Vasquez Jr. Cinema Crazed,2009-04-29,True,0.7,"A heartfelt, beautiful film, and one of the be..."
4,critic,tt0308644,/critic/urban-cinefile-critics/,Urban Cinefile Critics Urban Cinefile,2008-10-18,True,NaN,"Like Big Fish and The Neverending Story, this ..."


In [7]:
movie_awards_df.head()

,award_id,award_category,movie_imdb_id,person_imdb_id,award_date,winner,award_name,person_name
0,ev0000003,Best Motion Picture of the Year,tt5580390,nm0868219,2018-03-04,True,Oscars,Guillermo del Toro
1,ev0000003,Best Motion Picture of the Year,tt5580390,nm0197703,2018-03-04,True,Oscars,J. Miles Dale
2,ev0000003,Best Motion Picture of the Year,tt4555426,nm0079677,2018-03-04,False,Oscars,Tim Bevan
3,ev0000003,Best Motion Picture of the Year,tt4555426,nm0271479,2018-03-04,False,Oscars,Eric Fellner
4,ev0000003,Best Motion Picture of the Year,tt4555426,nm0115537,2018-03-04,False,Oscars,Lisa Bruce


In [8]:
madf = movie_awards_df[movie_awards_df.award_category == CATEGORY].drop(['person_imdb_id',
                                                                         'person_name',
                                                                         'award_id',
                                                                         'award_name',
                                                                         'award_category'],
                                                                        axis=1).drop_duplicates()

In [9]:
df_ = pd.merge(madf, 
               movies_df.drop(['release_year', 'countries', 'box_office', 'rt_url'], axis=1),
               left_on='movie_imdb_id',
               right_on='imdb_id').drop('imdb_id', axis=1)

In [10]:
full_df = pd.merge(df_, reviews_df, on='movie_imdb_id').drop(['movie_imdb_id', 'type', 'reviewer_name'], axis=1)

In [11]:
full_df.head()

,award_date,winner,title,directors,actors,runtime,genres,imdb_rating,imdb_votes,rt_tomato_score,rt_audience_score,reviewer_url,review_date,fresh,original_score,review_text
0,2018-03-04,True,The Shape of Water,[Guillermo del Toro],"[Sally Hawkins, Michael Shannon, Richard Jenki...",123 min,"[Adventure, Drama, Fantasy, Romance, Thriller]",7.4,273393.0,92.0,73.0,/critic/kyle-pinion/,2019-01-26,True,NaN,While it still falls short of his earlier triu...
1,2018-03-04,True,The Shape of Water,[Guillermo del Toro],"[Sally Hawkins, Michael Shannon, Richard Jenki...",123 min,"[Adventure, Drama, Fantasy, Romance, Thriller]",7.4,273393.0,92.0,73.0,/critic/marshall-shaffer/,2019-01-23,True,NaN,If The Shape of Water were purely focused on E...
2,2018-03-04,True,The Shape of Water,[Guillermo del Toro],"[Sally Hawkins, Michael Shannon, Richard Jenki...",123 min,"[Adventure, Drama, Fantasy, Romance, Thriller]",7.4,273393.0,92.0,73.0,/critic/andrew-wyatt/,2019-01-19,True,0.785714,A crystallization of timeless themes with exhi...
3,2018-03-04,True,The Shape of Water,[Guillermo del Toro],"[Sally Hawkins, Michael Shannon, Richard Jenki...",123 min,"[Adventure, Drama, Fantasy, Romance, Thriller]",7.4,273393.0,92.0,73.0,/critic/orlando-maldonado/,2019-01-18,True,0.800000,Guillermo del Toro's latest ask us about toler...
4,2018-03-04,True,The Shape of Water,[Guillermo del Toro],"[Sally Hawkins, Michael Shannon, Richard Jenki...",123 min,"[Adventure, Drama, Fantasy, Romance, Thriller]",7.4,273393.0,92.0,73.0,/critic/nathanael-hood/,2018-12-27,True,1.000000,Neil Gaiman once wrote that America is no plac...


In [12]:
df = full_df[full_df.award_date > full_df.review_date][[
    'award_date', 'winner', 'title', 'reviewer_url', 'fresh', 'original_score'
]]

In [13]:
df['year'] = df.apply(lambda x: x.award_date.year, axis=1)

In [14]:
df = df.drop('award_date', axis=1)

In [15]:
fmean = df.groupby('title', as_index=False)['fresh'].mean().rename(columns={'fresh': 'fresh_mean'})
smean = df[df.original_score.notnull()].groupby('title', as_index=False)['original_score'].mean().rename(columns={'original_score': 'score_mean'})

In [16]:
df2 = df[['winner', 'title', 'year']].drop_duplicates()
df3 = pd.merge(df2, fmean)
df3 = pd.merge(df3, smean)

In [19]:
df3.head()

,winner,title,year,fresh_mean,score_mean
0,True,The Shape of Water,2018,0.923529,0.827654
1,False,Darkest Hour,2018,0.858871,0.736528
2,False,The Post,2018,0.875776,0.785936
3,False,Dunkirk,2018,0.923077,0.856953
4,False,Lady Bird,2018,0.986755,0.860492


## Winner percentage in years 2000-2018 is 17%

In [51]:
len(df3[df3.year >= 2000][df3.winner == True]) / len(df3[df3.year >= 2000][df3.winner == False])

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


0.16964285714285715

## Check if score_mean predicts the winner

In [23]:
df_year_max = df3.groupby(['year', 'winner'], as_index=False)['score_mean'].max()

In [47]:
def winner_bigger(x, col):
    s_true, s_false = -1, -1
    for s, w in zip(x[col], x['winner']):
        if w:
            s_true = s
        else:
            s_false = s
    return s_true > s_false

In [37]:
pos_df = df_year_max[df_year_max.year >= 2000].groupby('year').apply(lambda x: winner_bigger(x, 'score_mean')).reset_index(name='positive')

In [44]:
len(pos_df[pos_df.positive == True]) / 19

0.42105263157894735

### <font color=red> In 42% of cases score_mean predicted the winner </color>

## Check if fresh_mean predicts the winner

In [48]:
df_year_max = df3.groupby(['year', 'winner'], as_index=False)['fresh_mean'].max()

In [49]:
pos_df = df_year_max[df_year_max.year >= 2000].groupby('year').apply(lambda x: winner_bigger(x, 'fresh_mean')).reset_index(name='positive')

In [50]:
len(pos_df[pos_df.positive == True]) / 19

0.2631578947368421

### Only in 26% cases fresh_mean predicted the winner